In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from numpy import array
from nltk.corpus import stopwords
from collections import Counter
from scipy.sparse import csr_matrix
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

from nltk import ngrams


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#TODO: Lemmatize correctly, this lemmatizer is really basic, it doesn't recognize verbs and nouns
sw = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()] + stopwords.words('french') + stopwords.words('english')
lemmatizer = FrenchLefffLemmatizer()
stemmer = nltk.stem.snowball.FrenchStemmer()

#stem, lemmatize, remove punctuation, and return a string
def process(article):
    res = ""
    for word in word_tokenize(article):
        if not word in sw+string.punctuation.split():
            if res == "": #if first word
                res += stemmer.stem(lemmatizer.lemmatize(word.lower()))
            else:
                res += " " + stemmer.stem(lemmatizer.lemmatize(word.lower()))
    return res


In [3]:
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

#Let's process our corpus, and determine a limit to split it in partitions
limit = 0
usable = []
for i in range(0, len(data['abstract']), 1):
    if not isinstance(data['abstract'][i], float): #if not null, empty, or whatever (so if there is a abstract)
        usable.append(process(data['abstract'][i]))
        if data['year'][i] <= 2010:
            limit += 1
    
print("nombre d'articles =", len(usable))
print("limit =", limit)

usable

nombre d'articles = 1096
limit = 553


["cet plateform object permettr citoyen d'analyserpar tweet polit d'éven spécif france.pour cas l'élect présidentiel 2017 , # idéo2017 analys quasitemp réel messag candidat , fourn principal caractérist , l'usag lexiqu polit comparaison candidat .",
 "la classif crois ( co-clustering ) techniqu d'extrair structuresous-jacent exist lign colon d'un tabl don form bloc . plusieur appliqu utilisent techniqu , algorithm co-clustering actuel passent l'échel . une approch utilis succes méthod modl , optimis criter vraisembl régularis . cependent , taill import , méthod atteint limit . dan articl , présenton nouvel algorithm co-clustering niveau , compt tenu criter modl trait efficac don grand taill , pouv mémoir . nos expérient montrent l'approch propos gagn temp calcul produis solut qualit .",
 "la recommand point d'intérêt devenu caractéristiqueessentiel réseau social géo-localis accompagnél'émergent échang massif don digital . cependantl faibl densit point d'intérêt visit utilis rendentl pr

<1096x124051 sparse matrix of type '<class 'numpy.float64'>'
	with 179071 stored elements in Compressed Sparse Row format>

In [5]:
nb_concepts = 30

min_gram = 1
max_gram = 3

cleandocs = []
for t in usable:
    doc = []
    for n in range(min_gram, max_gram+1):
        for gram in ngrams(t.split(), n):
            doc.append(" ".join(gram))
    cleandocs.append(doc)

corpus = []
first = True
if first:
    dictionary = corpora.Dictionary(cleandocs)
    first = False
for doc in cleandocs:
    newVec = dictionary.doc2bow(doc)
    corpus.append(newVec)
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
lsi = models.LsiModel(corpus_tfidf, num_topics=nb_concepts, id2word=dictionary)
corpus_lsi = lsi[corpus_tfidf]

i=0
for doc in corpus_lsi:
    print("document number ", i)
    i+=1
    print(doc)

document number  0
[(0, 0.025709019142108054), (1, 0.000107648390631929), (2, -0.00779755696711679), (3, -0.014101282152832366), (4, -0.028693313370810235), (5, -0.007370314222496888), (6, -0.0027077962103715337), (7, 0.010496031255771435), (8, -0.002524263786177544), (9, -0.014464823065113019), (10, -0.002350396088631528), (11, 0.003292515106848117), (12, 0.026802159816976038), (13, 0.0037387263006565096), (14, 0.022319390841275814), (15, 0.006074207243843374), (16, 0.015916791486687457), (17, -0.015718265214205357), (18, -0.03444273238067296), (19, 0.04371819297018485), (20, -0.0042479278572202985), (21, -0.011340334330584394), (22, 0.025176217116746304), (23, -0.0038190500621741964), (24, 0.03231739561157785), (25, 0.01021066749991219), (26, 0.028632826324362235), (27, -0.021030625460616934), (28, 0.007001927251823081), (29, 0.031239997292841673)]
document number  1
[(0, 0.09328985619514271), (1, 0.011483984052262877), (2, 0.00318695026386944), (3, 0.06111580779710513), (4, 0.021019

document number  71
[(0, 0.0958066086706319), (1, 0.005530789172844354), (2, -0.06516281384484746), (3, 0.007723293122252841), (4, 0.01975140926952143), (5, -0.006275878418285771), (6, 0.012032345871935817), (7, 0.035083881145532854), (8, 0.009964336813276324), (9, -0.03989169758483206), (10, -0.03738212373293813), (11, -0.007578723037339552), (12, -0.04819792797205791), (13, -0.006452673333724413), (14, 0.03532938337234186), (15, -0.0373248356898395), (16, 0.011254002692181917), (17, -0.023868859442223365), (18, -0.010209215167336678), (19, -0.02507852904634951), (20, -0.03659854109734479), (21, 0.012986677582756767), (22, -0.02903612351004716), (23, -0.011688514127168578), (24, -0.026625891678220763), (25, -0.04382223553401359), (26, -0.013785397623392955), (27, 0.022081169076974848), (28, 0.011245654738739697), (29, 0.012889320808764605)]
document number  72
[(0, 0.02121237192205675), (1, -0.08987216694947359), (2, -0.023118275896840682), (3, 0.005832498679433074), (4, -0.0025509876

[(0, 0.08067425188031994), (1, 0.008449887624934855), (2, -0.003929119540860821), (3, 0.0015683660709817503), (4, -0.010243936160591648), (5, 0.026213578065110044), (6, 0.035039200834698), (7, 0.00329754634161979), (8, 0.024060754292077685), (9, 0.04417174232526191), (10, -0.06705007250428058), (11, -0.032281112093192156), (12, -0.04038977433274692), (13, -0.013570493559333176), (14, 0.020945196477931048), (15, -0.04989985921068477), (16, -0.013181366846741243), (17, -0.013141563403770871), (18, 0.057899998128372016), (19, -0.026578389570155948), (20, 0.022366551778456104), (21, 0.03577986666737534), (22, 0.024615920438360132), (23, -0.050145317822331475), (24, -0.04129909388257477), (25, -0.0747800525228397), (26, -0.0022957411120529347), (27, -0.03705517413215791), (28, 0.0067379994120639955), (29, -0.013407567631582084)]
document number  175
[(0, 0.07745940156580815), (1, -0.0217528646902414), (2, -0.018527253949525792), (3, 0.016092407147408568), (4, -0.0037533423154080293), (5, 0.

document number  229
[(0, 0.06348446973864165), (1, 0.02404422612092936), (2, -0.010948692958477491), (3, 0.0058344206803139275), (4, -0.007971829412193203), (5, -0.010803311140151456), (6, -0.016960138930546043), (7, 0.0636254691419896), (8, 0.03133593129620931), (9, -0.027751696420998787), (10, -0.0403915664556351), (11, 0.02843982511281479), (12, -0.0016851663197153268), (13, -0.0017419853332341292), (14, 0.028623114421992613), (15, 0.0035707910649523554), (16, 0.057806744558703695), (17, -0.03220627416632226), (18, 0.03017491501118873), (19, -0.027068190195533533), (20, -0.010767339635885375), (21, 0.016469346749822204), (22, 0.04331941342905495), (23, 0.046044298224567404), (24, 0.02501417043670048), (25, 0.023806130089624532), (26, -0.00893666729427769), (27, -0.03548431276033422), (28, 0.06507330709654326), (29, -0.004054666728488276)]
document number  230
[(0, 0.01842523624878108), (1, -0.11533136188881955), (2, 0.0239709820967288), (3, -0.015485980511465715), (4, 0.00603057840

document number  325
[(0, 0.07887218310501347), (1, 0.008192722301339913), (2, 0.0267852634156826), (3, 0.14757817187363276), (4, -0.0026378453950588665), (5, 0.06792170472279853), (6, -0.10858107207132722), (7, -0.006265766273693088), (8, 0.0025746185388764572), (9, 0.0636814046647587), (10, -0.04974428637804804), (11, 0.02600435533759007), (12, -0.036429564212002566), (13, -0.06442530641781312), (14, -0.009991867094766608), (15, 0.029009414564353412), (16, -0.006783642265478428), (17, -0.002936729730356793), (18, 0.042346200742537546), (19, 0.04786325850816792), (20, 0.08975227886428506), (21, 0.059461564422576614), (22, 0.05405662584433614), (23, -0.027267743225119687), (24, -0.04356772474976265), (25, -0.020511189796825143), (26, 0.008370784802695258), (27, 0.08533253341060795), (28, -0.031004036286390108), (29, -0.07516545706472184)]
document number  326
[(0, 0.07735296506830304), (1, 0.009555696784100043), (2, 0.005400064447084805), (3, 0.013756607793540954), (4, -0.0036549068403

...

[(0, 0.07818488073968655), (1, 0.010157694726154931), (2, -0.014230881961314409), (3, -0.038089728837798306), (4, -0.0029146064458471094), (5, -0.021722506862768956), (6, 0.01391437111320357), (7, -0.05574696464047669), (8, 0.013338576390490723), (9, -0.025558928479028638), (10, 0.005103333870992823), (11, 0.028266675302488328), (12, -0.037565184486601864), (13, 0.023713769481819767), (14, -0.06519225651419468), (15, -0.010008300952062541), (16, 0.04093580387122044), (17, 0.026781001505625575), (18, -0.006966187741702663), (19, -0.025356426881272664), (20, -0.057625900569612566), (21, -0.04013849884739747), (22, 0.02757251356888108), (23, -0.030591752615114844), (24, -0.03756641414148494), (25, 0.05038702150641473), (26, -0.05143323946683219), (27, -0.004631124839094821), (28, -0.017096079490060555), (29, 0.006606990512730072)]
document number  880
[(0, 0.09409216366033275), (1, 0.021898993722632987), (2, 0.029879422241770023), (3, 0.07612191507469478), (4, -0.03284921075351794), (5, -

document number  955
[(0, 0.0700061167800861), (1, 0.003966218165448198), (2, 0.0022348879233661106), (3, 0.030722821035169202), (4, 0.016341229314974056), (5, -0.00663716883752884), (6, -0.019167633966170217), (7, 0.029535752820893848), (8, 0.006816404472350141), (9, -0.004552314602053751), (10, 0.003109540509611518), (11, -0.021076560337539588), (12, -0.020299913499722322), (13, 0.011091046245723933), (14, -0.020577502258866592), (15, -0.07306473492644708), (16, 0.0269696497446806), (17, 0.020558842573871937), (18, -0.036554185373441496), (19, -0.036609287958229736), (20, 0.009677097867591406), (21, -0.012399472631915447), (22, 0.02248729038759067), (23, 0.005508192352028548), (24, -0.0035269152784848587), (25, -0.010327689965654646), (26, 0.05582958214988689), (27, 0.01750950930914713), (28, -0.005773769542902059), (29, -0.04504494951921979)]
document number  956
[(0, 0.08389060749272689), (1, 0.03295321620869489), (2, -0.008860194816346334), (3, -0.004516811831192973), (4, -0.09382

document number  992
[(0, 0.07054382716664986), (1, 0.0028452420128044797), (2, 0.008898111884411272), (3, -0.025309495823031228), (4, 0.015074004615921505), (5, -0.021861118539117475), (6, -0.03619848557264124), (7, 0.024928961999820618), (8, 0.03919741616890221), (9, -0.05582160193555209), (10, 0.00812832952499224), (11, -0.037850718849201805), (12, 0.019417160283259915), (13, 0.008031593823362075), (14, -0.02596998514173877), (15, 0.004649082474505304), (16, -0.021230296872621214), (17, 0.05454091215857542), (18, -0.03704974257512563), (19, -0.022667105977993342), (20, 0.04598385948256657), (21, 0.029255713405072614), (22, 0.04823655501046735), (23, 0.04550980738499973), (24, -0.04412567362278023), (25, 0.035313912931339037), (26, -0.012147659537588921), (27, -0.004437213563717431), (28, 0.006673720488099581), (29, -0.04729716423971141)]
document number  993
[(0, 0.10788423786729606), (1, 0.025882672759175107), (2, -0.010298516581078446), (3, 0.012708736818872784), (4, -0.0652723821

document number  1024
[(0, 0.07440324759987756), (1, -0.004764994428945773), (2, -0.008496558952046068), (3, -0.03727960140059926), (4, 0.016486975682947956), (5, 0.00911490030009393), (6, 0.009533573888610177), (7, 0.028241799962296348), (8, -0.011230102641863133), (9, 0.02852070856773191), (10, -0.027063592839034638), (11, -0.018975066922774987), (12, 0.026912922150549464), (13, 0.04582575946348724), (14, -0.01242834277118407), (15, 0.005430492381346526), (16, 0.020869334539365205), (17, -0.038718234676219675), (18, -0.008812852357293753), (19, -0.05157224481783573), (20, -0.0127332949292151), (21, 0.04456987583117835), (22, -0.034704518715936064), (23, -0.013507024188430677), (24, 0.033820819077945326), (25, -0.024036552271143017), (26, 0.050138048741569324), (27, -0.01281167170346402), (28, -0.011896510206831468), (29, 0.02135897883949305)]
document number  1025
[(0, 0.0775363462639097), (1, 0.005611967217482877), (2, -0.018657035069619233), (3, -0.01113151853590577), (4, 0.0283294

document number  1092
[(0, 0.07523259247170996), (1, 0.010676382655927578), (2, 0.003516602746021693), (3, 0.010733917096370758), (4, -0.047630826170662464), (5, -0.019629532336635995), (6, 0.04127612920057646), (7, -0.05097897740401743), (8, 0.0008145476321760199), (9, -0.0347314922166994), (10, -0.016231537787879856), (11, 0.05323683636195306), (12, 0.003957935647229458), (13, -0.049310892717826814), (14, -0.00620503446406669), (15, 0.02584466071571636), (16, 0.03891874802923857), (17, -0.01137542001095928), (18, 0.005426573903337395), (19, -0.009221769184096569), (20, -0.0009697529520725274), (21, 0.03829465601016212), (22, -0.004199902467881925), (23, -0.014284227301858849), (24, 0.0050398376455244994), (25, 0.02060216748523789), (26, 0.022322378605778282), (27, -0.04501890653447269), (28, 0.01139715643454119), (29, -0.01528601423638619)]
document number  1093
[(0, 0.1207445245911362), (1, 0.0200938157127597), (2, 0.02582917614973834), (3, -0.04878603368923447), (4, 0.0314909175831

In [6]:
clusters = []
tresh = 0.03

for i in range(0, nb_concepts):
    dic = {}
    num_doc = 0
    for doc in corpus_lsi:
        if abs(doc[i][1]) > tresh:
            dic[num_doc] = doc[i][1]
        num_doc+=1
    clusters.append(dic)
clusters

[{1: 0.09328985619514271,
  2: 0.07133014968095255,
  3: 0.06543365168740412,
  4: 0.07460626859127431,
  5: 0.06019247577118048,
  6: 0.08002603359632868,
  7: 0.05839593424909585,
  8: 0.05867333507350808,
  9: 0.060552236186615115,
  10: 0.10170339683196354,
  11: 0.06871016464404844,
  12: 0.07661734616682667,
  13: 0.0855159476221539,
  14: 0.08407220838647794,
  15: 0.07482776268653557,
  16: 0.1104500013925226,
  17: 0.07811189985611953,
  18: 0.08367795698235372,
  19: 0.037512484907348105,
  20: 0.04997592165365922,
  21: 0.06742175406888329,
  22: 0.07525899319335093,
  23: 0.05541241008184396,
  24: 0.055118729818863475,
  25: 0.06397960716789027,
  26: 0.06047883799404874,
  27: 0.06099604811519977,
  28: 0.039296995160896776,
  29: 0.07841414090359612,
  30: 0.03783351610611218,
  31: 0.04160872950070297,
  32: 0.06177394974720806,
  33: 0.07269834668705887,
  34: 0.07183195534885066,
  35: 0.06120583058138495,
  36: 0.06832420258011244,
  37: 0.04419120984632005,
  38: 0.

In [7]:

vectorizer = TfidfVectorizer(stop_words=sw, use_idf=True, ngram_range=(min_gram, max_gram))
tfidf = vectorizer.fit_transform(usable)
len(tfidf.toarray()[0])


124051

In [18]:
clusters[3]

{1: 0.06111580779710513,
 3: 0.03480455642818129,
 4: -0.04067223269372427,
 10: 0.04047747759768309,
 12: -0.047213142679579025,
 14: -0.049954279385191466,
 15: -0.040110816821523496,
 17: -0.03299126621123591,
 22: -0.04520143740684836,
 27: -0.04551812906746865,
 30: -0.03847268095559865,
 34: 0.03435585841756518,
 39: 0.042700663855626146,
 44: -0.038148833477929495,
 45: 0.058037696557860986,
 50: 0.10078927807727024,
 52: 0.031467772335776065,
 53: -0.04047876581437665,
 54: -0.049002477329150536,
 55: -0.03266107957846569,
 56: -0.03356215657389721,
 60: 0.07142085883460236,
 66: 0.05145175111582607,
 68: 0.0405946264666508,
 69: 0.05481725045386539,
 70: -0.03475535437927716,
 73: 0.05054632824538067,
 77: -0.036712326924876526,
 78: -0.04441394443891995,
 81: -0.030046710464327313,
 83: -0.06839290346883528,
 85: -0.06639980599245535,
 86: 0.03158585589952374,
 93: 0.04069091258933757,
 95: 0.030431890399442302,
 99: 0.08818821583582424,
 100: 0.04107697441380512,
 104: 0.040

In [20]:
nb_labels_by_cluster = 5

# Let's labelize our clusters
# For this, we will use the tfidf matrix

vectorizer = TfidfVectorizer(stop_words=sw, use_idf=True, ngram_range=(min_gram, max_gram))
tfidf = vectorizer.fit_transform(usable)

# We can access the value in the tfidf using:
#tfidf.toarray().item(num_doc, num_word)
# To know the number of the word searched, we will use:
#vectorizer.vocabulary_[word]

# take less than 8h to compute x)
labels = []
for clus in clusters:

    first_arti = True
    for article in clus:
        link = abs(clus[article])
        if first_arti:
            coef_list = (tfidf.toarray()[article] * link)
            first = False
        else:
            # the more an article have a high coeficient, the more he is implied in the labeling step
            coef_list += (tfidf.toarray()[article] * link)
    # Now we have coef_list filled by every coeficient in the multiple tfidf
    # Let's find the best ones, to finally get the labels
    res = dict(zip(vectorizer.get_feature_names(), coef_list))
    
    labels.append(Counter(res).most_common(nb_labels_by_cluster))

labels

# TODO: on observe beaucoup de labels identiques entre deux clusters
# Je pense que c'est parce que l'on a trop de clusters, mais j'aimerais en être sûr

[[('entrepôt', 0.008789085599219175),
  ('articl examinon', 0.004400520967129649),
  ('articl examinon possibil', 0.004400520967129649),
  ('conceptuel entrepôt', 0.004400520967129649),
  ('conceptuel entrepôt propos', 0.004400520967129649)]]

#### Nous obtenons ici x clusters, représentés par des concepts grâce au LSA
#### Nous allons seuiller et étiquetter ces clusters

In [ ]:
#Here, I determine a tresh, to remove documents from clusters
#Thus, I tried to compute the X² of the values for each cluster. This X² was my tresh
#Yeah yeah yeah. I know, X² is a law, what I compute is just a value to compare to this law
#But I find this more pictured to talk about it this way
#PS: (don't trust this snippet, I think it's wrong)

partitions = []
tresh = 0

for lsa_part in [lsa1, lsa2]:
    # X² pour ne garder que les articles au dessus d'un certain seuil
    temp = 0
    sumConcept = 0
    for concept in transpose(lsa):
        for article in concept:
            sumConcept += article
            
            
            tresh += pow(article, 2)
    tresh /= (lsa_part.shape[0]*lsa_part.shape[1])
    

In [ ]:
def indexOf(matrixSrc, listSearched):
    if (len(matrixSrc[1]) != len(listSearched)):
        return -1
    pos = 0
    for l in matrixSrc:
        thesame = True
        for i in range(0, len(listSearched), 1):
            if l[i] != listSearched[i]:
                thesame = False
                break
        if thesame:
            return pos
        pos+=1
    return -2
a = [[1, 2, 3], [4, 2, 1], [2, 3, 5], [3, 2, 1]]
b = [3,2,1]
indexOf(a, b)

In [ ]:
#I know, transpose exists in numpy
#However, it didn't work with me because I got sparse matrix, so I was deseperate and I wrote my own function
def transpose(matrix):
    transp = []
    for j in range(0, matrix.shape[1], 1):
        l = []
        for i in range(0, matrix.shape[0], 1):
            l.append(matrix[i][j])
        transp.append(l)
    return transp


In [ ]:
def mostCommons(dico, tresh):
    res = []
    for name in dico:
        if dico[name] >= tresh:
            res.append(name)
    return res

test = {"abc": 3, "trucmuch":5, "doge":0}
mostCommons(test, 3)

In [ ]:
#IMPORTANT!! The good way to get an element from document-term matrix
tfidf.toarray().item(0, 0)